In [3]:
import numpy as np
import math

In [4]:
def SE2_theta(theta: float=0):
    return np.matrix([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta), np.cos(theta), 0],
        [0, 0, 1]
    ])

def SE2_xy(x, y):
    m = np.array([
        [1, 0, x],
        [0, 1, y],
        [0, 0, 1]
    ])
    return m

def ik(theta1, theta2):
    a1 = 1.0
    a2 = 1.0
    dist_sq = x**2 + y**2

    cos_theta2 = (dist_sq - a1**2 - a2**2)/(2*a1*a2)

    cos_theta2 = max(min(cos_theta2,1),-1)
    sin_theta2 = np.sqrt(max(0,1 - cos_theta2**2))

    theta2 = math.atan2(sin_theta2, cos_theta2)

    theta1 = math.atan2(y, x) - math.atan2(a2*sin_theta2, a1 + a2*cos_theta2)

    return (theta1, theta2)

def fk(theta1, theta2):
    a1 = 1.0
    a2 = 1.0
    x = a1 * np.cos(theta1) + a2 * np.cos(theta1 + theta2)
    y = a1 * np.sin(theta1) + a2 * np.sin(theta1 + theta2)
    th = theta1 + theta2
    return (x, y, th)


In [5]:
SE2_xy(1, .25) @ np.array([.5, .5, 1])

array([1.5 , 0.75, 1.  ])

In [6]:
np.linalg.inv(SE2_xy(1, .25)) @ np.array([.5, .5, 1])

array([-0.5 ,  0.25,  1.  ])

In [7]:
(SE2_xy(1, 0.25) @ SE2_theta(np.radians(45))) @ (np.array([.5, .5, 1]))

matrix([[1.        , 0.95710678, 1.        ]])

In [8]:
np.linalg.inv(SE2_xy(1, 0.25) @ SE2_theta(np.radians(45))) @ np.array([.5, .5, 1])

matrix([[-0.1767767 ,  0.53033009,  1.        ]])

In [10]:
angles = ((0, 90), (90, 90), (90, -90), (-180, 180))
for theta_1, theta_2 in angles:
    t1, t2 = np.radians(theta_1), np.radians(theta_2)   
    print("theta_1 = {:.4f}, theta2 = {:.4f} => {}".format(t1, t2, fk(t1, t2)))

theta_1 = 0.0000, theta2 = 1.5708 => (1.0, 1.0, 1.5707963267948966)
theta_1 = 1.5708, theta2 = 1.5708 => (-0.9999999999999999, 1.0000000000000002, 3.141592653589793)
theta_1 = 1.5708, theta2 = -1.5708 => (1.0, 1.0, 0.0)
theta_1 = -3.1416, theta2 = 3.1416 => (0.0, -1.2246467991473532e-16, 0.0)


In [14]:
points = ((1, 1), (1, -1), (-1, 1), (-1, -1), (2, 1), (2, 0), (0, 2), (-2, 0))
for x, y in points:
    print(f"({x}, {y}) =>", ("{:.2f} | " * 2).format(*ik(x, y)))

(1, 1) => 0.00 | 1.57 | 
(1, -1) => -1.57 | 1.57 | 
(-1, 1) => 1.57 | 1.57 | 
(-1, -1) => -3.14 | 1.57 | 
(2, 1) => 0.46 | 0.00 | 
(2, 0) => 0.00 | 0.00 | 
(0, 2) => 1.57 | 0.00 | 
(-2, 0) => 3.14 | 0.00 | 
